# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

import pandas as pd
import os
import sys
from glob import glob
import dask.dataframe as dd


sys.path.append(os.getenv('SRC_DIR'))
sys.path.append('../../05_src')

from utils.logger import get_logger
_logs = get_logger(__name__)


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [2]:
import random

stock_files = glob(os.path.join(os.getenv('SRC_DIR'), "data/prices_csv/stocks/*.csv"))

random.seed(42)
stock_files = random.sample(stock_files, 60)

dt_list = []
for s_file in stock_files:
    _logs.info(f"Reading file: {s_file}")
    dt = pd.read_csv(s_file).assign(
        source = os.path.basename(s_file),
        ticker = os.path.basename(s_file).replace('.csv', ''),
        Date = lambda x: pd.to_datetime(x['Date']),
    )
    dt_list.append(dt)
stock_prices = pd.concat(dt_list, axis = 0, ignore_index = True)

2025-06-08 13:32:47,615, 65293645.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/HCM.csv
2025-06-08 13:32:47,642, 65293645.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/ATHM.csv
2025-06-08 13:32:47,648, 65293645.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/RILYP.csv
2025-06-08 13:32:47,651, 65293645.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/ALDX.csv
2025-06-08 13:32:47,654, 65293645.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/JHG.csv
2025-06-08 13:32:47,657, 65293645.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/CHRS.csv
2025-06-08 13:32:47,673, 65293645.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/BKTI.csv
2025-06-08 13:32:47,700, 65293645.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/REFR.csv
2025-06-08 13:32:47,706, 65293645.py, 10, INFO, Reading file: ../../05_src/data/prices_csv/stocks/KEN.csv
2025-06-08 13:32:47,711, 65293645.py, 1

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [3]:
PRICE_DATA = os.getenv("PRICE_DATA")

In [4]:
#Add lags for variables Close and Adj_Close.
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

dd_shift = dd_px.assign(
    Close_lag_1 = dd_px.groupby('ticker')['Close'].shift(1),
    Adj_Close_lag_1 = dd_px.groupby('ticker')['Adj Close'].shift(1),
    
    returns = lambda x: (x['Close'] / x['Close_lag_1']) - 1,
    hi_lo_range = dd_px['High'] - dd_px['Low'],
    
    
)
dd_feat = dd_px.assign(
    Close_lag_1 = dd_px.groupby('ticker')['Close'].shift(1),
    Adj_Close_lag_1 = dd_px.groupby('ticker')['Adj Close'].shift(1),
    # Add the returns calculation
    returns = lambda x: (x['Close'] / x['Close_lag_1']) - 1,
    # Add the hi_lo_range
    hi_lo_range = dd_px['High'] - dd_px['Low']
)




/var/folders/7j/sh_gsz3s72b0xzcpbczx9l440000gn/T/ipykernel_77348/3102904485.py:6: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  Close_lag_1 = dd_px.groupby('ticker')['Close'].shift(1),
/var/folders/7j/sh_gsz3s72b0xzcpbczx9l440000gn/T/ipykernel_77348/3102904485.py:7: UserWarning: `meta` is not specified, inferred from partial data.
Please provide `meta` if the result is unexpected.
  Before: .shift(func)
  After:  .shift(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .shift(func, meta=('x', 'f8'))            for series result

  Adj_Close_lag_1 = dd_px.groupby('ticker')['Adj Close'].shift(1),
/var/folders/7j/sh_gsz3s72b0xzcpbczx9l440000gn/T/ipykernel_77348/3102904485.py:15: UserWarning: `meta` is not specified, inferred from partial

In [5]:
print(dd_feat.dtypes)

Date                datetime64[ns]
Open                       float64
High                       float64
Low                        float64
Close                      float64
Adj Close                  float64
Volume                     float64
source             string[pyarrow]
Year                         int32
Close_lag_1                float64
Adj_Close_lag_1            float64
returns                    float64
hi_lo_range                float64
dtype: object


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [6]:
# Write your code below.

df_feat = dd_feat.compute()

In [7]:
print(f"Type: {type(df_feat)}")
print(f"Shape: {df_feat.shape}")
print(f"Columns: {df_feat.columns.tolist()}")

Type: <class 'pandas.core.frame.DataFrame'>
Shape: (230273, 13)
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source', 'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns', 'hi_lo_range']


In [8]:
# Add a new feature containing the moving average of returns using a window of 10 days
df_feat['returns_ma_10'] = df_feat.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(0, drop=True)

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not necessary to use pandas. We could use Dask to do it, and since our DataFrame was already a Dask DataFrame, it makes more sense to apply the changes to the Dask DataFrame rather than convert it to pandas. For large databases, it is better to use Dask, but our dataset here is not that large and we can technically do it in pandas. However, since it is already in Dask, it just makes sense not to convert it and do it in Dask instead.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.